# EDA
---

In [1]:
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
books = pd.read_csv('data/books.csv')
rating = pd.read_csv('data/ratings.csv')

In [3]:
books['book_title'].duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
149565     True
149566    False
149567    False
149568    False
149569    False
Name: book_title, Length: 149570, dtype: bool

---
## 고전 명서는 인기가 많을까? 평가는 어떨까?
고전 명서: 
같은 제목, 작가로 다양한 출판사, 연도 등이 존재하는 책

In [5]:
classic = books['book_title'].value_counts()[:100].index

In [8]:
top_dup_book = list(books.query('book_title in @classic').isbn)

In [11]:
rating.query('isbn == @top_dup_book').rating.mean()

7.62625

고전 명서의 평균 평점은 약 7.6점이다

## 파레토 법칙: 전체 소비의 80%는 20%의 인기있는 아이템에서 발생한다.
---

In [12]:
PARETO = int(len(rating['isbn'].unique()) * 0.2)

In [13]:
top_20 = rating['isbn'].value_counts()[:PARETO]
print(f'ratio of top 20: {sum(top_20) / len(rating)}')

ratio of top 3020: 0.6112974461774149


전체 소비의 61%가 인기순 상위 20%에서 발생한다.

## 콜드 스타트
---

In [16]:
test = pd.read_csv('data/test_ratings.csv')
train = pd.read_csv('data/train_ratings.csv')
books = pd.read_csv('data/books_1.0.csv')
both_rating = pd.concat([train, test], axis=0)

In [34]:
test_user = test.user_id.unique()
train_user = train.user_id.unique()
test_item = test.isbn.unique()
train_item = train.isbn.unique()

In [36]:
test.query('isbn not in @train_item')

,user_id,isbn,rating
7,11676,0671870432,0
64,14,0689821166,0
420,53,0245542957,0
538,87,0375509038,0
929,176,1900850303,0
...,...,...,...
76694,278543,1576734218,0
76695,278563,3492223710,0
76696,278633,1896095186,0
76697,278668,8408044079,0


Test 셋에서 처음 등장하는 유저의 수

In [37]:
import numpy as np
cold_user = (set(test_user) - set(train_user))
print(f'number of cold start user: {len(cold_user)}')
print(f'cold start ratio: {len(cold_user) / len(test)}')

np.save('no_train_user',np.array(cold_user))

number of cold start user: 8266
cold start ratio: 0.10777193966023026


In [19]:
train.user_id.value_counts()

11676     5520
98391     4560
189835    1503
153662    1496
23902      956
          ... 
54908        1
236844       1
44531        1
93836        1
278713       1
Name: user_id, Length: 59803, dtype: int64

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

plt.bar(np.arange(len(train_user[:100])),train.user_id.value_counts().iloc[:100])
plt.show()

In [21]:
print(f"mean of user's ratings: {train.user_id.value_counts().values.mean()}")
print(f"median of user's ratings: {np.median(train.user_id.value_counts().values)}")
print(f"median of user's ratings: {stats.mode(train.user_id.value_counts().values)}")

mean of user's ratings: 5.130093808002943
median of user's ratings: 1.0
median of user's ratings: ModeResult(mode=array([1]), count=array([34617]))


In [22]:
users = pd.read_csv('data/users_f_location_1.1.csv')

In [23]:
len(set(test_user) - set(users.user_id.unique()))

0

구매를 한번도 하지 않았어도 유저에 대한 정보(`location`, `age`)는 존재한다

### Cold user & Cold item

In [24]:
train_user_vc = train.user_id.value_counts()
print(f'전체 사람 수: {len(train.user_id.unique())}')
print(f'5권 이하로 읽은 사람의 수(비율):{train_user_vc.where(train_user_vc < 5).count()}, {train_user_vc.where(train_user_vc < 5).count() / len(train.user_id.unique()):.2f}')

전체 사람 수: 59803
5권 이하로 읽은 사람의 수(비율):49119, 0.82


In [25]:
train_book_vc = train.isbn.value_counts()
print(f'전체 책 수: {len(train.isbn.unique())}')
print(f'5번 이하로 읽은 책의 수(비율):{train_book_vc.where(train_book_vc < 5).count()}, {train_book_vc.where(train_book_vc < 5).count() / len(train.isbn.unique()):.2f}')

전체 책 수: 129777
5번 이하로 읽은 책의 수(비율):119456, 0.92


In [26]:
cold_user_id = train_user_vc.where(train_user_vc < 5).dropna().index
cold_book_isbn = train_book_vc.where(train_book_vc < 5).dropna().index

print(f'콜드 유저 수: {len(cold_user_id)}')
print(f'콜드 아이템 수: {len(cold_book_isbn)}')
print(f"콜드 유저가 콜드 아이템을 평가한 수: {len(train.query('user_id in @cold_user_id and isbn in @cold_book_isbn'))}")
print(f'전체 평가 수: {len(train)}')

콜드 유저 수: 49119
콜드 아이템 수: 119456
콜드 유저가 콜드 아이템을 평가한 수: 34804
전체 평가 수: 306795


### Warm user & Warm item

In [27]:
train_user_vc = train.user_id.value_counts()
print(f'전체 사람 수: {len(train.user_id.unique())}')
print(f'5권 이하로 읽은 사람의 수(비율):{train_user_vc.where(train_user_vc >= 5).count()}, {train_user_vc.where(train_user_vc >= 5).count() / len(train.user_id.unique()):.2f}')

전체 사람 수: 59803
5권 이하로 읽은 사람의 수(비율):10684, 0.18


In [28]:
train_book_vc = train.isbn.value_counts()
print(f'전체 책 수: {len(train.isbn.unique())}')
print(f'5번 이하로 읽은 책의 수(비율):{train_book_vc.where(train_book_vc >= 5).count()}, {train_book_vc.where(train_book_vc >= 5).count() / len(train.isbn.unique()):.2f}')

전체 책 수: 129777
5번 이하로 읽은 책의 수(비율):10321, 0.08


In [29]:
warm_user_id = train_user_vc.where(train_user_vc >= 5).dropna().index
warm_book_isbn = train_book_vc.where(train_book_vc >= 5).dropna().index

print(f'warm 유저 수: {len(warm_user_id)}')
print(f'warm 아이템 수: {len(warm_book_isbn)}')
print(f"warm 유저가 warm 아이템을 평가한 수: {len(train.query('user_id in @warm_user_id and isbn in @warm_book_isbn'))}")
print(f'전체 평가 수: {len(train)}')

warm 유저 수: 10684
warm 아이템 수: 10321
warm 유저가 warm 아이템을 평가한 수: 101866
전체 평가 수: 306795


In [30]:
np.save('data/cold_book_isbn',np.array(cold_book_isbn))
np.save('data/cold_user_id', np.array(cold_user_id))
np.save('data/warm_book_isbn',np.array( warm_book_isbn))
np.save('data/warm_user_id',np.array(warm_user_id))

In [31]:
print(type(train.user_id.unique()[0]))

<class 'numpy.int64'>
